# RAGAS Evaluation Notebook (Clean)

This notebook evaluates **baseline vs experiment** RAG pipelines using RAGAS and saves **summary/detail** outputs per run.

In [16]:
# %pip install datasets

In [19]:
# %pip install ragas

## 0) Environment & Paths

In [1]:
import os, sys, json, time
from pathlib import Path

import pandas as pd

# === Project paths (edit if needed) ===
PROJECT_ROOT = Path(r"C:\ai\source\chatbot_app")
MODULES_DIR  = PROJECT_ROOT / "modules"
ENV_PATH     = PROJECT_ROOT / ".env"

# Add modules dir so we can import rag_module.py
sys.path.insert(0, str(MODULES_DIR))

# Load .env (optional but recommended)
try:
    from dotenv import load_dotenv
    load_dotenv(ENV_PATH)
    print(f"✅ Loaded .env from: {ENV_PATH}")
except Exception as e:
    print("⚠️ dotenv load failed (can continue):", e)

print("✅ sys.path[0] =", sys.path[0])
print("✅ rag_module.py exists:", (MODULES_DIR / "rag_module.py").exists())


✅ Loaded .env from: C:\ai\source\chatbot_app\.env
✅ sys.path[0] = C:\ai\source\chatbot_app\modules
✅ rag_module.py exists: True


## 1) Load testset (JSONL)

In [6]:
TESTSET_JSONL = PROJECT_ROOT / "ragas_testset_one.jsonl"  # change if needed
assert TESTSET_JSONL.exists(), f"❌ JSONL not found: {TESTSET_JSONL}"

rows = []
with open(TESTSET_JSONL, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        rows.append(json.loads(line))

print("✅ rows:", len(rows))
print("✅ keys example:", rows[0].keys())
pd.DataFrame(rows[:3])


✅ rows: 1
✅ keys example: dict_keys(['question', 'ground_truth'])


,question,ground_truth
0,"전입신고·확정일자 했는데, 확정일자부 내용까지 중요한가요?","네, 중요합니다. 확정일자를 받았다는 사실만으로 끝나는 것이 아니라, 확정일자부에 ..."


## 2) Define baseline & experiment configs

- Keep **base_cfg** stable.
- Only put **changed knobs** in `exp_cfg = replace(base_cfg, ...)`.

In [7]:
from dataclasses import replace
from rag_module import RAGConfig

# =========================
# Base config (edit as needed)
# =========================
base_cfg = RAGConfig(
    # ---- LLM ----
    normalize_model="solar-pro2",
    generation_model="gpt-4o-mini",
    temperature=0.1,
    normalize_temperature=0.0,

    # ---- Embedding ----
    embedding_backend="upstage",
    embedding_model="solar-embedding-1-large-passage",

    # ---- Dense Retrieval ----
    k_law=5,
    k_rule=5,
    k_case=3,
    search_multiplier=2,

    # ---- BM25 / Sparse ----
    enable_bm25=True,

    # ---- Rerank ----
    enable_rerank=True,
    rerank_threshold=0.2,
    rerank_max_documents=20,

    # ---- Output trimming ----
    bm25_max_doc_chars=2400,
    rerank_doc_max_chars=2400,
)

# =========================
# Experiment config (only diffs here)
# =========================
exp_cfg = replace(
    base_cfg,
    hybrid_dense_weight=0.5,
    hybrid_sparse_weight=0.5,
    hybrid_sparse_title_ratio=0.45,
)

base_cfg, exp_cfg


(RAGConfig(normalize_model='solar-pro2', generation_model='gpt-4o-mini', temperature=0.1, normalize_temperature=0.0, embedding_backend='upstage', embedding_model='solar-embedding-1-large-passage', k_law=5, k_rule=5, k_case=3, search_multiplier=2, enable_bm25=True, sparse_mode='auto', sparse_k_law=None, sparse_k_rule=None, sparse_k_case=None, bm25_algorithm='okapi', bm25_k1=1.5, bm25_b=0.75, bm25_use_kiwi=True, bm25_max_doc_chars=2400, enable_bm25_title=True, bm25_title_field='title', bm25_title_max_chars=512, hybrid_sparse_title_ratio=0.35, hybrid_fusion='rrf', hybrid_dense_weight=0.6, hybrid_sparse_weight=0.4, rrf_k=60, enable_rerank=True, rerank_threshold=0.2, rerank_model='rerank-multilingual-v3.0', rerank_max_documents=20, rerank_doc_max_chars=2400, case_candidate_k=40, case_expand_top_n=None, case_context_top_k=50, dedupe_key_fields=('chunk_id', 'id')),
 RAGConfig(normalize_model='solar-pro2', generation_model='gpt-4o-mini', temperature=0.1, normalize_temperature=0.0, embedding_ba

## 3) Build pipelines

In [8]:
from rag_module import create_pipeline

base_pipe = create_pipeline(config=base_cfg)
exp_pipe  = create_pipeline(config=exp_cfg)

print("✅ pipelines ready")


2026-01-30 17:34:22,659 - rag_module - INFO - 🔗 Pinecone 3중 인덱스 연결 중...
2026-01-30 17:34:22,661 - rag_module - INFO - ✅ [Law / Rule / Case] 3개 인덱스 로드 완료!
2026-01-30 17:34:23,047 - rag_module - INFO - ✅ Kiwi 토크나이저 사용 (BM25)
2026-01-30 17:34:24,334 - rag_module - INFO - 🔗 Pinecone 3중 인덱스 연결 중...
2026-01-30 17:34:24,334 - rag_module - INFO - ✅ [Law / Rule / Case] 3개 인덱스 로드 완료!
2026-01-30 17:34:24,740 - rag_module - INFO - ✅ Kiwi 토크나이저 사용 (BM25)


✅ pipelines ready


## 4) (Optional) Quick trace sanity check

In [9]:
# If your rag_module exposes a trace / debug method, call it here.
# Otherwise you can skip this cell.

# Example (adjust to your actual API):
# ans, trace = base_pipe.answer_with_trace("테스트 질문 ...")
# display(trace)

print("ℹ️ Skip or customize depending on your pipeline API.")


ℹ️ Skip or customize depending on your pipeline API.


## 5) Build RAGAS samples from your pipeline outputs

This converts each testset row into the RAGAS format:
- `question`
- `answer`
- `contexts` (list[str])
- `ground_truth`

In [12]:
def shrink_contexts(ctxs, max_chars=2400, max_contexts=30):
    out = []
    for c in (ctxs or []):
        if c is None:
            continue
        s = str(c).strip()
        if not s:
            continue
        out.append(s[:max_chars])
        if len(out) >= max_contexts:
            break
    return out

def row_get_ground_truth(r: dict):
    return r.get("ground_truth") or r.get("reference") or r.get("gt") or r.get("answer")

def run_pipe_to_samples(pipe, rows, max_chars=2400, max_contexts=30, limit=None):
    samples = []
    n = len(rows) if limit is None else min(limit, len(rows))

    for i in range(n):
        r = rows[i]
        q = r.get("question") or r.get("query")
        if not q:
            continue

        # ✅ 네 파이프라인은 이걸로 호출해야 함
        out = pipe.answer_with_trace(q)

        # out이 dict일 수도 있고, (answer, ctxs, trace) 튜플일 수도 있어서 안전 처리
        ans, ctxs, trace = "", [], None

        if isinstance(out, dict):
            ans = out.get("answer") or out.get("result") or out.get("output") or out.get("text") or ""
            ctxs = out.get("contexts") or out.get("context") or out.get("docs") or []
            trace = out.get("trace") or out.get("debug") or out.get("meta")
        elif isinstance(out, tuple):
            # 흔한 패턴들 대응
            if len(out) == 3:
                ans, ctxs, trace = out
            elif len(out) == 2:
                ans, ctxs = out
            elif len(out) == 1:
                ans = out[0]
        else:
            ans = str(out)

        samples.append({
            "question": q,
            "answer": ans or "",
            "contexts": shrink_contexts(ctxs, max_chars=max_chars, max_contexts=max_contexts),
            "ground_truth": row_get_ground_truth(r) or "",
            "_trace": trace,  # ✅ trace도 같이 보관(원하면 저장 가능)
        })

    return samples

# ✅ 스모크 테스트: 1문제만
BASE_SAMPLES = run_pipe_to_samples(base_pipe, rows, max_chars=2400, max_contexts=30, limit=1)
EXP_SAMPLES  = run_pipe_to_samples(exp_pipe,  rows, max_chars=2400, max_contexts=30, limit=1)

print("✅ BASE_SAMPLES:", len(BASE_SAMPLES))
print("✅ EXP_SAMPLES :", len(EXP_SAMPLES))
pd.DataFrame([{k:v for k,v in BASE_SAMPLES[0].items() if k != "_trace"}])

2026-01-30 17:36:21,369 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-01-30 17:36:21,383 - rag_module - INFO - 🔄 표준화된 질문: 주민등록·확정일자(확정일자) 했는데, 확정일자부(확정일자부) 내용까지 중요한가요?
2026-01-30 17:36:21,384 - rag_module - INFO - 🔍 [Hybrid Retrieval] query='주민등록·확정일자(확정일자) 했는데, 확정일자부(확정일자부) 내용까지 중요한가요?'
2026-01-30 17:36:21,898 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-30 17:36:24,071 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-30 17:36:26,341 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-30 17:36:31,165 - httpx - INFO - HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
2026-01-30 17:36:31,187 - rag_module - INFO - 📌 Rerank selected=2 (threshold=0.2)
2026-01-30 17:36:31,188 - rag_module - INFO - 🤖 답변 생성 중...
2026-01-30 17:36:38,139 - httpx - 

✅ BASE_SAMPLES: 1
✅ EXP_SAMPLES : 1


,question,answer,contexts,ground_truth
0,"전입신고·확정일자 했는데, 확정일자부 내용까지 중요한가요?",A. 확정일자부의 내용은 매우 중요합니다.\n\nB. 지금 당장 할 일\n - ...,[page_content='① 제3조의2제2항의 확정일자는 주택 소재지의 읍ㆍ면사무...,"네, 중요합니다. 확정일자를 받았다는 사실만으로 끝나는 것이 아니라, 확정일자부에 ..."


## 6) RAGAS evaluation (prepared cell)

- Creates per-sample detail dataframe (when supported by your RAGAS version)
- Creates summary dataframe (mean over samples)
- Keeps timing metrics

In [22]:
# ============================================================
# RAGAS compare + clean saving (no deprecated imports/wrappers)
# - metrics: ragas.metrics.collections
# - LLM: ragas.llms.llm_factory (no LangchainLLMWrapper)
# - saves: summary/detail/meta + config.json + samples_base/exp.jsonl + trace.jsonl
# - run dir: auto-increment + timestamp
# ============================================================

import time, json, re
from pathlib import Path
from datetime import datetime

import pandas as pd
from datasets import Dataset

from openai import OpenAI
from ragas import evaluate
from ragas.llms import llm_factory
from ragas.metrics.collections import ContextPrecision, ContextRecall, Faithfulness, AnswerRelevancy


# ----------------------------
# LLM + METRICS (non-deprecated)
# ----------------------------
client = OpenAI()  # uses OPENAI_API_KEY env var
llm = llm_factory("gpt-4o-mini", client=client)

METRICS = [
    ContextPrecision(llm=llm),
    ContextRecall(llm=llm),
    Faithfulness(llm=llm),
]


# ----------------------------
# utils
# ----------------------------
def _json_safe(obj):
    """Make config/meta safe to dump to json."""
    try:
        json.dumps(obj, ensure_ascii=False)
        return obj
    except TypeError:
        if hasattr(obj, "model_dump"):
            return obj.model_dump()
        if hasattr(obj, "dict"):
            return obj.dict()
        if hasattr(obj, "__dict__"):
            return obj.__dict__
        return str(obj)

def _write_json(path: Path, data):
    path.write_text(json.dumps(_json_safe(data), ensure_ascii=False, indent=2), encoding="utf-8")

def _write_jsonl(path: Path, rows):
    with path.open("w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(_json_safe(r), ensure_ascii=False) + "\n")

def _next_run_dir(project_root: Path, prefix: str):
    runs_root = Path(project_root) / "results" / "ragas_runs"
    runs_root.mkdir(parents=True, exist_ok=True)

    # Find existing {prefix}_0001_* directories
    pat = re.compile(rf"^{re.escape(prefix)}_(\d{{4}})_")
    nums = []
    for p in runs_root.iterdir():
        if p.is_dir():
            m = pat.match(p.name)
            if m:
                nums.append(int(m.group(1)))
    next_idx = (max(nums) + 1) if nums else 1

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = runs_root / f"{prefix}_{next_idx:04d}_{ts}"
    run_dir.mkdir(parents=True, exist_ok=False)
    return run_dir, next_idx, ts


# ----------------------------
# core eval
# ----------------------------
def eval_ragas_with_details(samples, run_tag: str):
    ds = Dataset.from_list(samples)

    t0 = time.time()
    res = evaluate(dataset=ds, metrics=METRICS)
    eval_sec = time.time() - t0

    t1 = time.time()
    detail_df = res.to_pandas() if hasattr(res, "to_pandas") else pd.DataFrame()
    to_pandas_sec = time.time() - t1

    samples_df = pd.DataFrame(samples)

    # merge per-sample metrics back onto samples
    if len(detail_df) == len(samples_df) and len(detail_df) > 0:
        out_detail = pd.concat(
            [samples_df.reset_index(drop=True), detail_df.reset_index(drop=True)],
            axis=1
        )
    else:
        out_detail = samples_df.copy()

    out_detail["run_tag"] = run_tag
    out_detail["eval_seconds"] = round(eval_sec, 3)
    out_detail["to_pandas_seconds"] = round(to_pandas_sec, 3)

    # summary (mean of numeric metric columns if available)
    summary = {}
    if len(detail_df) > 0:
        summary = detail_df.mean(numeric_only=True).to_dict()
    elif isinstance(res, dict):
        summary = {k: float(v) for k, v in res.items() if isinstance(v, (int, float))}

    summary["run_tag"] = run_tag
    summary["eval_seconds"] = round(eval_sec, 3)
    summary["to_pandas_seconds"] = round(to_pandas_sec, 3)

    return res, out_detail, pd.DataFrame([summary])


# ----------------------------
# compare + save (clean)
# ----------------------------
def run_compare_and_save(
    base_samples,
    exp_samples,
    project_root: Path,
    prefix="ragas_compare",
    base_cfg=None,
    exp_cfg=None,
):
    base_res, base_detail_df, base_summary_df = eval_ragas_with_details(base_samples, "baseline")
    exp_res,  exp_detail_df,  exp_summary_df  = eval_ragas_with_details(exp_samples,  "experiment")

    summary_df = pd.concat([base_summary_df, exp_summary_df], ignore_index=True)
    detail_df  = pd.concat([base_detail_df,  exp_detail_df],  ignore_index=True)

    run_dir, run_id, ts = _next_run_dir(project_root, prefix)

    # fixed filenames inside run dir (clean)
    out_summary = run_dir / "summary.csv"
    out_detail  = run_dir / "detail.csv"
    out_meta    = run_dir / "meta.json"
    out_config  = run_dir / "config.json"
    out_base_in = run_dir / "samples_base.jsonl"
    out_exp_in  = run_dir / "samples_exp.jsonl"
    out_trace   = run_dir / "trace.jsonl"  # best-effort trace

    summary_df.to_csv(out_summary, index=False, encoding="utf-8-sig")
    detail_df.to_csv(out_detail, index=False, encoding="utf-8-sig")

    # config snapshot (best-effort)
    cfg_payload = {
        "base_cfg": _json_safe(base_cfg) if base_cfg is not None else None,
        "exp_cfg":  _json_safe(exp_cfg)  if exp_cfg  is not None else None,
        "llm": {"model": "gpt-4o-mini"},
        "metrics": ["ContextPrecision", "ContextRecall", "Faithfulness", "AnswerRelevancy"],
    }
    _write_json(out_config, cfg_payload)

    # input snapshots
    _write_jsonl(out_base_in, base_samples)
    _write_jsonl(out_exp_in,  exp_samples)

    # trace snapshot (best-effort)
    # - if your samples include _trace, keep it
    trace_rows = []
    for s in list(base_samples) + list(exp_samples):
        if isinstance(s, dict) and ("_trace" in s):
            trace_rows.append({"question": s.get("question"), "run_tag": s.get("run_tag"), "_trace": s.get("_trace")})
    if not trace_rows:
        # fallback: store minimal fields from detail
        cols = [c for c in ["run_tag", "question", "eval_seconds"] if c in detail_df.columns]
        trace_rows = detail_df[cols].to_dict(orient="records") if cols else []
    _write_jsonl(out_trace, trace_rows)

    meta = {
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "run_id": run_id,
        "timestamp": ts,
        "run_dir": str(run_dir),
        "prefix": prefix,
        "n_base_samples": len(base_samples),
        "n_exp_samples": len(exp_samples),
        "saved": {
            "summary": str(out_summary),
            "detail": str(out_detail),
            "meta": str(out_meta),
            "config": str(out_config),
            "samples_base": str(out_base_in),
            "samples_exp": str(out_exp_in),
            "trace": str(out_trace),
        },
    }
    _write_json(out_meta, meta)

    return {
        "base_res": base_res,
        "exp_res": exp_res,
        "summary_df": summary_df,
        "detail_df": detail_df,
        "run_dir": run_dir,
        "out_summary": out_summary,
        "out_detail": out_detail,
        "out_meta": out_meta,
        "out_config": out_config,
        "out_samples_base": out_base_in,
        "out_samples_exp": out_exp_in,
        "out_trace": out_trace,
        "run_id": run_id,
    }

## 7) Run + compare + save

In [ ]:
import time, re, json
from pathlib import Path
from datetime import datetime

import pandas as pd
from datasets import Dataset

from openai import OpenAI
from ragas import evaluate
from ragas.llms import llm_factory

# ----------------------------
# LLM (non-deprecated)
# ----------------------------
client = OpenAI()
llm = llm_factory("gpt-4o-mini", client=client)

# ----------------------------
# METRICS (version-tolerant)
# - 1순위: 구버전/호환 높은 "metric objects" (context_precision 등)
# - 2순위: 신버전 "classes" (ContextPrecision 등)
# ----------------------------
def build_metrics():
    # try older/object style first (most compatible)
    try:
        from ragas.metrics import context_precision, context_recall, faithfulness
        return [context_precision, context_recall, faithfulness]
    except Exception:
        pass

    # fallback to class style
    from ragas.metrics.collections import ContextPrecision, ContextRecall, Faithfulness
    return [ContextPrecision(), ContextRecall(), Faithfulness()]

METRICS = build_metrics()

def eval_ragas_with_details(samples, run_tag: str):
    ds = Dataset.from_list(samples)

    t0 = time.time()
    res = evaluate(dataset=ds, metrics=METRICS, llm=llm)  # ✅ llm은 여기로
    eval_sec = time.time() - t0

    t1 = time.time()
    detail_df = res.to_pandas() if hasattr(res, "to_pandas") else pd.DataFrame()
    to_pandas_sec = time.time() - t1

    samples_df = pd.DataFrame(samples)

    # merge
    if len(detail_df) == len(samples_df) and len(detail_df) > 0:
        out_detail = pd.concat(
            [samples_df.reset_index(drop=True), detail_df.reset_index(drop=True)],
            axis=1
        )
    else:
        out_detail = samples_df.copy()

    out_detail["run_tag"] = run_tag
    out_detail["eval_seconds"] = round(eval_sec, 3)
    out_detail["to_pandas_seconds"] = round(to_pandas_sec, 3)

    # summary
    summary = {}
    if len(detail_df) > 0:
        summary = detail_df.mean(numeric_only=True).to_dict()
    elif isinstance(res, dict):
        summary = {k: float(v) for k, v in res.items() if isinstance(v, (int, float))}

    summary["run_tag"] = run_tag
    summary["eval_seconds"] = round(eval_sec, 3)
    summary["to_pandas_seconds"] = round(to_pandas_sec, 3)

    return res, out_detail, pd.DataFrame([summary])

def _next_run_dir(project_root: Path, prefix: str):
    runs_root = Path(project_root) / "results" / "ragas_runs"
    runs_root.mkdir(parents=True, exist_ok=True)

    pat = re.compile(rf"^{re.escape(prefix)}_(\d{{4}})_")
    nums = []
    for p in runs_root.iterdir():
        if p.is_dir():
            m = pat.match(p.name)
            if m:
                nums.append(int(m.group(1)))
    next_id = (max(nums) + 1) if nums else 1

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = runs_root / f"{prefix}_{next_id:04d}_{ts}"
    run_dir.mkdir(parents=True, exist_ok=False)
    return run_dir

def run_compare_and_save(base_samples, exp_samples, project_root: Path, prefix="ragas_compare"):
    base_res, base_detail_df, base_summary_df = eval_ragas_with_details(base_samples, "baseline")
    exp_res,  exp_detail_df,  exp_summary_df  = eval_ragas_with_details(exp_samples,  "experiment")

    summary_df = pd.concat([base_summary_df, exp_summary_df], ignore_index=True)
    detail_df  = pd.concat([base_detail_df,  exp_detail_df],  ignore_index=True)

    run_dir = _next_run_dir(project_root, prefix)

    # 기본 산출물 (항상 같은 파일명)
    out_summary = run_dir / "summary.csv"
    out_detail  = run_dir / "detail.csv"
    out_meta    = run_dir / "meta.json"

    summary_df.to_csv(out_summary, index=False, encoding="utf-8-sig")
    detail_df.to_csv(out_detail, index=False, encoding="utf-8-sig")

    meta = {
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "run_dir": str(run_dir),
        "prefix": prefix,
        "n_base_samples": len(base_samples),
        "n_exp_samples": len(exp_samples),
        "metrics": [getattr(m, "__name__", m.__class__.__name__) for m in METRICS],
    }
    out_meta.write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")

    return {
        "base_res": base_res,
        "exp_res": exp_res,
        "summary_df": summary_df,
        "detail_df": detail_df,
        "run_dir": str(run_dir),
        "out_summary": str(out_summary),
        "out_detail": str(out_detail),
        "out_meta": str(out_meta),
    }

# ----------------------------
# RUN (스모크 테스트면 limit=1 샘플로 먼저)
# ----------------------------
result = run_compare_and_save(
    base_samples=BASE_SAMPLES,
    exp_samples=EXP_SAMPLES,
    project_root=PROJECT_ROOT,
    prefix="ragas_compare",
)

print("✅ run_dir:", result["run_dir"])
print("✅ saved:", result["out_summary"], result["out_detail"], result["out_meta"])

C:\Users\Admin\AppData\Local\Temp\ipykernel_13696\818566082.py:26: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import context_precision, context_recall, faithfulness
C:\Users\Admin\AppData\Local\Temp\ipykernel_13696\818566082.py:26: DeprecationWarning: Importing context_recall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_recall
  from ragas.metrics import context_precision, context_recall, faithfulness
C:\Users\Admin\AppData\Local\Temp\ipykernel_13696\818566082.py:26: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import f

In [ ]:
from pathlib import Path
from datetime import datetime
import json

# ----------------------------
# helpers
# ----------------------------
def _json_safe(obj):
    try:
        json.dumps(obj, ensure_ascii=False)
        return obj
    except TypeError:
        if hasattr(obj, "model_dump"):
            return obj.model_dump()
        if hasattr(obj, "dict"):
            return obj.dict()
        if hasattr(obj, "__dict__"):
            return obj.__dict__
        return str(obj)

def _write_json(path: Path, data):
    path.write_text(json.dumps(_json_safe(data), ensure_ascii=False, indent=2), encoding="utf-8")

def _write_jsonl(path: Path, rows):
    with path.open("w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(_json_safe(r), ensure_ascii=False) + "\n")

def _df_to_jsonl_rows(df, prefer_cols):
    cols = [c for c in prefer_cols if c in df.columns]
    if cols:
        df = df[cols].copy()
    return df.to_dict(orient="records"), cols

# ----------------------------
# run_dir from result (셀1 실행 후)
# ----------------------------
run_dir = Path(result["run_dir"])
run_dir.mkdir(parents=True, exist_ok=True)

# ----------------------------
# 1) config snapshot
# ----------------------------
cfg_payload = {}

# base_cfg/exp_cfg가 노트북 전역에 있으면 저장 (없으면 None)
cfg_payload["base_cfg"] = _json_safe(globals().get("base_cfg")) if "base_cfg" in globals() else None
cfg_payload["exp_cfg"]  = _json_safe(globals().get("exp_cfg"))  if "exp_cfg"  in globals() else None

# llm/metrics 정보도 같이 남김(재현성)
cfg_payload["llm"] = {"model": "gpt-4o-mini"}
cfg_payload["created_at"] = datetime.now().isoformat()

_write_json(run_dir / "config.json", cfg_payload)

# ----------------------------
# 2) input samples snapshot
# ----------------------------
_write_jsonl(run_dir / "samples_base.jsonl", BASE_SAMPLES)
_write_jsonl(run_dir / "samples_exp.jsonl",  EXP_SAMPLES)

# ----------------------------
# 3) trace snapshot (from detail_df)
# ----------------------------
detail_df = result["detail_df"]
if not hasattr(detail_df, "columns"):
    # 혹시 detail_df가 df가 아니라면 안전 처리
    import pandas as pd
    detail_df = pd.DataFrame(detail_df)

trace_cols_priority = [
    "id", "sample_id",
    "question", "normalized_question", "normalized_query",
    "answer", "ground_truth", "reference",
    "contexts",
    "_trace",  # ✅ 너가 answer_with_trace에서 넣어둔 경우
    "retrieved_doc_ids", "retrieved_docs", "retrieval_scores",
    "rerank_selected_ids", "rerank_scores",
    "final_context_ids", "final_contexts",
    "latency_ms", "latency_sec",
    "run_tag",
]

trace_rows, used_cols = _df_to_jsonl_rows(detail_df, trace_cols_priority)
_write_jsonl(run_dir / "trace.jsonl", trace_rows)

print("✅ extra saved config :", run_dir / "config.json")
print("✅ extra saved samples:", run_dir / "samples_base.jsonl", "and", run_dir / "samples_exp.jsonl")
print("✅ extra saved trace  :", run_dir / "trace.jsonl")
print("✅ trace columns used :", used_cols)